In [28]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [29]:
# enter details of your AML workspace
subscription_id = "31093e7f-3cdc-4559-9889-69d7eebc31ce"
resource_group = "jcrg"
workspace = "jc-aml-ws"

In [30]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [31]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "ep-ipmodel-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an endpoint for IP model",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [32]:
#create endpoint in workspace
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://ep-ipmodel-01031912307940.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://ep-ipmodel-01031912307940.eastus2.inference.ml.azure.com/swagger.json', 'name': 'ep-ipmodel-01031912307940', 'description': 'this is an endpoint for IP model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/resourcegroups/jcrg/providers/microsoft.machinelearningservices/workspaces/jc-aml-ws/onlineendpoints/ep-ipmodel-01031912307940', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:7aa091ce-1fa5-4d20-8422-3130e4780d77:436fecaf-1dda-45f1-ae8f-4d5d788ebafa?api-version=2022-02-01-preview'}, 'id': '/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/resourceGroups/jcrg/provi

Configure online deployment:
A deployment is a set of resources required for hosting the model that does the actual inferencing

In [39]:
model = Model(path="../ip_endpoint_tutorial/model/PM2_MultiGrade_STFImodel.pkl")
# env = Environment(
#     conda_file="../model-1/environment/conda.yml",
#     image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
# )

env = ml_client.environments.get(name="ip-env", version="1")

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../ip_endpoint_tutorial/onlinescoring", scoring_script="scoring.py"
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

In [40]:
#create deployment in workspace
ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint ep-ipmodel-01031912307940 exists
Uploading onlinescoring (0.0 MBs): 100%|██████████| 2164/2164 [00:00<00:00, 52591.93it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


................................................................................

ManagedOnlineDeployment({'private_network_connection': False, 'data_collector': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'ep-ipmodel-01031912307940', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/od:7aa091ce-1fa5-4d20-8422-3130e4780d77:6a351e9b-44fc-4e34-bfd4-7946835996eb?api-version=2022-02-01-preview'}, 'id': '/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/resourceGroups/jcrg/providers/Microsoft.MachineLearningServices/workspaces/jc-aml-ws/onlineEndpoints/ep-ipmodel-01031912307940/deployments/blue', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/aml-instance/code/Users/juichiache/ip_endpoint_tutorial', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fd954b91c

In [35]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://ep-ipmodel-01031912307940.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://ep-ipmodel-01031912307940.eastus2.inference.ml.azure.com/swagger.json', 'name': 'ep-ipmodel-01031912307940', 'description': 'this is an endpoint for IP model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/resourcegroups/jcrg/providers/microsoft.machinelearningservices/workspaces/jc-aml-ws/onlineendpoints/ep-ipmodel-01031912307940', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:7aa091ce-1fa5-4d20-8422-3130e4780d77:535aa8c9-c3cf-4b2d-9156-66dd7113eade?api-version=2022-02-01-preview'}, 'id': '/subscriptions/31093e7f-3cdc-4559-9889-69d7eebc31ce/resourceGroups/jcrg/provi

In [41]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="../ip_endpoint_tutorial/test_data.json",
)

'[21.080642060616853]'